In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import os


In [2]:
os.chdir('C:\\analise_ frebre_ amarela\\asset') #entra na pasta do dastaset
print(os.getcwd())

C:\analise_ frebre_ amarela\asset


1 Entendimento dos Dados
| Objetivo - Identificar as informações relevantes que serão necessárias para compor o objetivo do projeto, reconhecido como a confecção de um painel interativo (Dashboard).

Revisão dos dados
| Objetivo - Identificar as colunas de maior relevância, com base nas informações recebida e definidas como análises 

Identificação:

* ID: Identificador sequencial único 

* MACRORREG_LPI: Sigla da macrorregião do local provável de infecção 

* COD_UF_LPI: Código IBGE da Unidade Federada do local provável de infecção.

* UF_LPI: Sigla da Unidade Federada do local provável de infecção.

* COD_MUN_LPI: Código IBGE do município do local provável de infecção.
  
* SEXO IDADE: Sexo do indivíduo Idade do indivíduo.
   
* DT_IS: Data de início dos sintomas do indivíduo (dd/mm/aaaa).
  
* SE_IS : Semana epidemiológica de início dos sintomas do indivíduo.

* MES_IS: Mês de início dos sintomas do indivíduo.

* ANO_IS: Ano de início dos sintomas do indivíduo.

* MONITORAMENTO_IS: Período de monitoramento de início dos sintomas do indivíduo.

* OBITO: Evolução para o óbito.

* DT_OBITO: Data do óbito (dd/mm/aaaa).



In [3]:
df = pd.read_csv("dataset.csv", encoding='latin-1', delimiter=';')
df.head()

,ID,MACRORREG_LPI,COD_UF_LPI,UF_LPI,COD_MUN_LPI,MUN_LPI,SEXO,IDADE,DT_IS,SE_IS,MES_IS,ANO_IS,MONITORAMENTO_IS,OBITO,DT_OBITO
0,1,N,14,RR,140005.0,ALTO ALEGRE,M,NaN,29/11/1994,48.0,11.0,1994,1994/1995,SIM,01/12/1994
1,2,N,14,RR,140045.0,PACARAIMA,M,19,19/02/1995,8.0,2.0,1995,1994/1995,NÃO,NaN
2,3,NE,21,MA,210060.0,AMARANTE DO MARANHÃO,M,32,01/04/1995,13.0,4.0,1995,1994/1995,IGN,NaN
3,4,NE,21,MA,210060.0,AMARANTE DO MARANHÃO,M,33,03/04/1995,14.0,4.0,1995,1994/1995,SIM,09/04/1995
4,5,N,15,PA,150270.0,CONCEIÇÃO DO ARAGUAIA,M,26,12/04/1995,15.0,4.0,1995,1994/1995,SIM,20/04/1995


In [4]:
#dropando as colunas que não serão utilizada
df.drop(columns=['MACRORREG_LPI', 'COD_UF_LPI', 'COD_MUN_LPI','DT_IS', 'SE_IS'], inplace=True)

In [5]:
#renomendando as colunas
df = df.rename(columns={
    'UF_LPI':'UF',
    'MUN_LPI': 'MUNICIPIO',
    'SE_IS': 'SEMANA_INICIO_SINTOMA',
    'MES_IS': 'MES_INICIO_SINTOMA',
    'ANO_IS': 'ANO_INICIO_SINTOMA',
    })

In [6]:
#identificar e mostrar valores nulos 
df_missing = (
  df.isna().sum()
  .to_frame('qtd_dadosfaltantes')
  .join((df.isna().sum() / df.shape[0]).to_frame('%_dadosfaltantes'))
  .sort_values('qtd_dadosfaltantes', ascending=False)
)

df_missing['%_dadosfaltantes'] = df_missing['%_dadosfaltantes'].apply(lambda x: f"{x:.2%}")

In [7]:
df_missing


,qtd_dadosfaltantes,%_dadosfaltantes
DT_OBITO,1789,64.63%
MES_INICIO_SINTOMA,11,0.40%
MUNICIPIO,6,0.22%
IDADE,2,0.07%
SEXO,1,0.04%
ID,0,0.00%
UF,0,0.00%
ANO_INICIO_SINTOMA,0,0.00%
MONITORAMENTO_IS,0,0.00%
OBITO,0,0.00%


In [8]:
#excluindo as linhas nulas
df = df.dropna()

In [9]:
#checando as valores nulos depois da exclusão
df.isnull().sum()

ID                    0
UF                    0
MUNICIPIO             0
SEXO                  0
IDADE                 0
MES_INICIO_SINTOMA    0
ANO_INICIO_SINTOMA    0
MONITORAMENTO_IS      0
OBITO                 0
DT_OBITO              0
dtype: int64

In [10]:
df.head()

,ID,UF,MUNICIPIO,SEXO,IDADE,MES_INICIO_SINTOMA,ANO_INICIO_SINTOMA,MONITORAMENTO_IS,OBITO,DT_OBITO
3,4,MA,AMARANTE DO MARANHÃO,M,33,4.0,1995,1994/1995,SIM,09/04/1995
4,5,PA,CONCEIÇÃO DO ARAGUAIA,M,26,4.0,1995,1994/1995,SIM,20/04/1995
5,6,AM,TAPAUÁ,M,29,1.0,1996,1995/1996,SIM,07/02/1996
6,7,AM,RIO PRETO DA EVA,M,38,1.0,1996,1995/1996,SIM,05/02/1996
7,8,AM,MANAQUIRI,M,46,1.0,1996,1995/1996,SIM,11/02/1996


In [14]:
df.dtypes

ID                      int64
UF                     object
MUNICIPIO              object
SEXO                   object
IDADE                  object
MES_INICIO_SINTOMA    float64
ANO_INICIO_SINTOMA      int64
MONITORAMENTO_IS       object
OBITO                  object
DT_OBITO               object
dtype: object

In [16]:
df['DT_OBITO'] = pd.to_datetime(df['DT_OBITO'], errors='coerce') #transforma objeto em datas

In [24]:
df['IDADE'] = pd.to_numeric(df['IDADE'], errors='coerce') #converter para numericos

In [37]:
#agrupando por estados 
obitos_UF = df.groupby('UF')['OBITO'].count(). reset_index()
obitos_UF.sort_values(by='OBITO', ascending=False)

,UF,OBITO
7,MG,367
17,SP,233
12,RJ,99
4,ES,82
5,GO,51
10,PA,37
1,AM,29
9,MT,17
18,TO,11
14,RR,11


In [34]:
obitos_UF = df.groupby('MUNICIPIO')['OBITO'].count().reset_index()
obitos_UF.sort_values(by='OBITO', ascending=False)

,MUNICIPIO,OBITO
201,MAIRIPORÃ,42
189,LADAINHA,19
24,ANGRA DOS REIS,16
32,ATIBAIA,16
142,GUARULHOS,14
...,...,...
17,AMARANTE DO MARANHÃO,1
16,AMARALINA,1
15,ALVORADA DE MINAS,1
14,ALVINÓPOLIS,1
